# Voclo Trainer - Google Colab

This notebook allows you to train a voice conversion model for voclo directly in Google Colab.

## Steps:
1. Upload your audio data
2. Configure training parameters
3. Train the model
4. Export to ONNX
5. Download the model


## 1. Install Dependencies


In [ ]:
# Install all required libraries
%pip install torch torchaudio librosa soundfile scipy numpy tqdm pyyaml pandas scikit-learn onnx onnxruntime onnxsim -q

# Check GPU
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")


## 2. Load voclo-trainer Code

Load project files using one of the following methods:
- Via GitHub (if you have a repository)
- Via Colab file manager
- Or use the inline version below


In [ ]:
# Create directory structure
import os
os.makedirs('voclo-trainer/models', exist_ok=True)
os.makedirs('voclo-trainer/utils', exist_ok=True)
os.makedirs('voclo-trainer/configs', exist_ok=True)
os.makedirs('data/train', exist_ok=True)
os.makedirs('data/val', exist_ok=True)
os.makedirs('checkpoints', exist_ok=True)

print("Directories created")

# If you have a GitHub repository, uncomment:
# !git clone https://github.com/your-username/voclo-trainer.git
# %cd voclo-trainer


## 3. Load Data

### Option A: Upload ZIP Archive from Computer


In [ ]:
from google.colab import files
import zipfile

# Upload ZIP archive with your audio files
# Structure should be:
# data.zip
#   train/
#     speaker1/
#       audio1.wav
#       audio2.wav
#   val/
#     speaker1/
#       audio1.wav

print("Click the button below to upload ZIP archive with data")
uploaded = files.upload()

# Extract
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall('.')
        print(f"Extracted: {filename}")
        os.remove(filename)  # Remove ZIP after extraction


### Option B: Load from Google Drive


In [ ]:
from google.colab import drive
import shutil

# Mount Google Drive
drive.mount('/content/drive')

# Specify path to your data in Drive
# For example: /content/drive/MyDrive/voclo_data/
drive_data_path = '/content/drive/MyDrive/voclo_data/'

# Copy data (if needed)
if os.path.exists(drive_data_path):
    if os.path.exists('data'):
        shutil.rmtree('data')
    shutil.copytree(drive_data_path, 'data')
    print(f"Data copied from {drive_data_path}")
else:
    print(f"Path {drive_data_path} not found. Check the path.")


### Verify Data


In [ ]:
import librosa
from pathlib import Path

# Check data structure
train_dir = Path('data/train')
val_dir = Path('data/val')

def count_audio_files(directory):
    count = 0
    total_duration = 0
    for ext in ['*.wav', '*.mp3', '*.flac']:
        for file in directory.rglob(ext):
            try:
                audio, sr = librosa.load(str(file), sr=None, duration=0.1)
                duration = len(audio) / sr if sr > 0 else 0
                total_duration += duration
                count += 1
            except:
                pass
    return count, total_duration

train_count, train_duration = count_audio_files(train_dir)
val_count, val_duration = count_audio_files(val_dir)

print(f"Training data:")
print(f"  Files: {train_count}")
print(f"  Total duration: {train_duration/60:.2f} minutes")
print(f"\nValidation data:")
print(f"  Files: {val_count}")
print(f"  Total duration: {val_duration/60:.2f} minutes")

if train_count == 0:
    print("\n⚠️ WARNING: Training data not found!")
    print("Make sure data is loaded in the correct structure.")


## 4. Configure


In [ ]:
import yaml

# Create training configuration
config = {
    'model': {
        'name': 'rvc',
        'sample_rate': 44100,
        'hop_length': 512,
        'n_fft': 2048,
        'n_mels': 80,
        'hidden_size': 256,
        'num_layers': 3,
        'num_speakers': 1,
        'embed_dim': 256,
        'f0_bin': 256,
        'f0_min': 50.0,
        'f0_max': 1100.0
    },
    'training': {
        'batch_size': 16,  # Reduce if out of memory
        'num_epochs': 100,
        'learning_rate': 0.0001,
        'weight_decay': 0.0001,
        'save_interval': 10,
        'validation_interval': 5,
        'gradient_clip': 1.0,
        'loss_weights': {
            'mel': 1.0,
            'recon': 1.0,
            'kl': 0.1,
            'f0': 0.5
        }
    },
    'data': {
        'train_dir': 'data/train',
        'val_dir': 'data/val',
        'segment_length': 8192,
        'num_workers': 2,  # For Colab
        'pin_memory': True,
        'augmentation': {
            'enabled': True,
            'pitch_shift_range': [-2, 2],
            'time_stretch_range': [0.9, 1.1],
            'noise_level': 0.01
        }
    },
    'logging': {
        'log_dir': 'logs',
        'tensorboard': False,  # Disabled for Colab
        'console_log_interval': 100
    }
}

# Save configuration
os.makedirs('configs', exist_ok=True)
with open('configs/colab_config.yaml', 'w') as f:
    yaml.dump(config, f, default_flow_style=False)

print("Configuration saved to configs/colab_config.yaml")
print(f"\nTraining parameters:")
print(f"  Batch size: {config['training']['batch_size']}")
print(f"  Epochs: {config['training']['num_epochs']}")
print(f"  Learning rate: {config['training']['learning_rate']}")


In [ ]:
# Если у вас есть репозиторий на GitHub:
# !git clone https://github.com/your-username/voclo-trainer.git
# %cd voclo-trainer

# Или загрузите файлы через файловый менеджер Colab (слева)
print("Загрузите файлы проекта через файловый менеджер или используйте git clone")
print("\nНеобходимые файлы:")
print("  - train.py")
print("  - export_onnx.py")
print("  - models/ (папка с моделями)")
print("  - utils/ (папка с утилитами)")


## 6. Обучение модели

⚠️ **ВНИМАНИЕ**: Обучение может занять несколько часов. Убедитесь, что у вас активна GPU сессия в Colab.


In [ ]:
# Запуск обучения
# Убедитесь, что все файлы проекта загружены

!python train.py --config configs/colab_config.yaml


## 7. Export to ONNX


In [ ]:
# Export best model
model_name = "my_character"  # Change to your character name

!python export_onnx.py --checkpoint checkpoints/best_model.pth --name {model_name} --output {model_name}_sr44100_hop512_nfft2048.onnx

print(f"\n✅ Model exported: {model_name}_sr44100_hop512_nfft2048.onnx")


## 8. Download Results


In [ ]:
from google.colab import files
import zipfile

# Download ONNX model
onnx_files = list(Path('.').glob('*.onnx'))
if onnx_files:
    for onnx_file in onnx_files:
        files.download(str(onnx_file))
        print(f"Downloaded: {onnx_file}")
else:
    print("ONNX files not found. Check export path.")

# Download all checkpoints (optional)
if os.path.exists('checkpoints'):
    checkpoint_files = list(Path('checkpoints').glob('*.pth'))
    if checkpoint_files:
        # Create ZIP archive
        zip_path = 'checkpoints.zip'
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file in checkpoint_files:
                zipf.write(file, file.name)
        files.download(zip_path)
        print(f"Downloaded checkpoint archive: {zip_path}")


### Save to Google Drive


In [ ]:
# Save results to Google Drive
drive_backup_path = '/content/drive/MyDrive/voclo_results/'
os.makedirs(drive_backup_path, exist_ok=True)

# Copy checkpoints
if os.path.exists('checkpoints'):
    import shutil
    backup_checkpoints = os.path.join(drive_backup_path, 'checkpoints')
    if os.path.exists(backup_checkpoints):
        shutil.rmtree(backup_checkpoints)
    shutil.copytree('checkpoints', backup_checkpoints)
    print(f"Checkpoints saved to {backup_checkpoints}")

# Copy ONNX models
onnx_files = list(Path('.').glob('*.onnx'))
for onnx_file in onnx_files:
    shutil.copy(onnx_file, drive_backup_path)
    print(f"Model saved: {onnx_file.name}")

print(f"\n✅ All results saved to {drive_backup_path}")


## Tips for Working in Colab

1. **GPU Session**: Make sure GPU is enabled (Runtime → Change runtime type → GPU)
2. **Timeout**: Colab may disconnect after inactivity. Use:
   - Periodic print() for activity
   - Colab Pro for longer sessions
3. **Memory**: If out of memory, reduce `batch_size` in configuration
4. **Saving**: Regularly save results to Google Drive
5. **Checkpoints**: Save checkpoints every few epochs

## Troubleshooting

- **Out of memory**: Reduce `batch_size` or `segment_length`
- **Session interrupted**: Load checkpoints and continue training with `--resume`
- **Slow training**: Make sure GPU is used, not CPU

Detailed guide: see [COLAB_GUIDE.md](COLAB_GUIDE.md)
